# PyPSA Outage Examples

Take some example outages on our Hawaii40 case. Notice how the load changes in the network

In [24]:
import pandas as pd
import io
import pypsa

In [25]:
# Linear powerflow .lpf() is an approximation of full powerflow. It may make be useful if have convergence issues
USE_LPF = False

In [26]:
def check_pf(info):
    converged = info.converged.any().any()
    max_error = info.error.max().max()
    print(f"Sim converged: {converged}")
    print(f"Max error: {max_error:.2e}")

    if ~converged:
        raise Exception("Sim didn't convert - results are garbage. Change to lpf()")

## Load the model

In [27]:
# If we run lpf() again, the old results stick around L48 which is inactive.
# We can start from new and completely discard the old results gone
network = pypsa.Network()
network.import_from_csv_folder('csv')
if USE_LPF:
    network.lpf() 
else:
    network.pf() 

INFO:pypsa.network.io:Imported network 'Unnamed Network' has buses, generators, lines, loads, shunt_impedances, transformers
INFO:pypsa.network.power_flow:Performing non-linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x000001A24A3487A0> for snapshots Index(['now'], dtype='object', name='snapshot')


## Take an outages and solve

Let's take an outage on 
```
L48 | 19 - 23 | PEARL CITY69 - WAIPAHU69 #1
```

Then check to see if the parallel lines, or any other line, overloads.
```
L49 | 19 - 23 | PEARL CITY69 - WAIPAHU69 #2
L50 | 19 - 23 | PEARL CITY69 - WAIPAHU69 #3
L51 | 19 - 23 | PEARL CITY69 - WAIPAHU69 #4
```

In [28]:
# Put the nominal ratings in their own dataframe for later
s_nom = network.lines[['s_nom']]
# s_nom.head()

In [29]:
# Get the nominal rating of the lines care about
network.lines[['s_nom']].loc[['L48', 'L49', 'L50', 'L51']]

,s_nom
name,
L48,96.0
L49,96.0
L50,96.0
L51,96.0


In [30]:
# Get the loading before the outage - should be about 77 MVA 
# Remember positive and negative is just the direction of the flow. 
# When we think about loading, we look at the magnitude
network.lines_t['p0'].loc['now'][['L48', 'L49', 'L50', 'L51']]

name
L48   -77.532753
L49   -77.532753
L50   -77.532753
L51   -77.532753
Name: now, dtype: float64

In [31]:
# Disable one of the lines
network.lines.loc["L48", "active"] = False
# make sure you clear the solve data for the disabled line 
# I'm sure there's a better way to do this, but I don't know how
# The old solve data will stay where it is unless you explicitly do something about it
network.lines_t['p0'].loc['now', 'L48'] = 0.0 # Clear old results 

# pf() sometimes has trouble converging - you can use lpf() if you want.
if USE_LPF:
    network.lpf() 
else:
    info = network.pf() # 
    check_pf(info)

# Now show the results 
network.lines_t['p0'].loc['now'][['L48', 'L49', 'L50', 'L51']]

INFO:pypsa.network.power_flow:Performing non-linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x000001A24A3DD7F0> for snapshots Index(['now'], dtype='object', name='snapshot')


Sim converged: True
Max error: 1.91e-07


name
L48     0.000000
L49   -98.551717
L50   -98.551717
L51   -98.551717
Name: now, dtype: float64

In [32]:
# Calculate the percent loading for everything in the network
# Note that now some of the lines are overloaded 
p0 = network.lines_t['p0'].loc['now']
p0.name = 'p0_now'
p0_df = pd.DataFrame(p0)
p0_df = p0_df['p0_now'].map(abs).round(0)

df = pd.merge(s_nom, p0_df, how='inner', left_index=True, right_index=True)
df['loading_frac'] = (1 - (df['s_nom'] - df['p0_now']) / df['s_nom']).round(2)
df.sort_values(by='loading_frac', ascending=False).head()

,s_nom,p0_now,loading_frac
name,,,
L51,96.0,99.0,1.03
L49,96.0,99.0,1.03
L50,96.0,99.0,1.03
L67,70.0,54.0,0.77
L20,86.0,59.0,0.69


## Take another outage and solve

Lets see how much worse things get if we disable another circuit. We call this an N-2 contingency.
Things should start to get bad for the paralell circuits (L50, L51)

In [33]:
# Now disabled another circuit - something will break for sure
network.lines.loc["L49", "active"] = False
network.lines_t['p0'].loc['now', 'L49'] = 0.0 # Clear old results
if USE_LPF:
    network.lpf() 
else:
    info = network.pf() # 
    check_pf(info) 

INFO:pypsa.network.power_flow:Performing non-linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x000001A24A464A10> for snapshots Index(['now'], dtype='object', name='snapshot')


Sim converged: True
Max error: 3.98e-07


In [34]:
network.lines_t['p0'].loc['now'][['L48', 'L49', 'L50', 'L51']]

name
L48      0.00000
L49      0.00000
L50   -135.18757
L51   -135.18757
Name: now, dtype: float64

In [35]:
p0 = network.lines_t['p0'].loc['now']
p0.name = 'p0_now'
p0_df = pd.DataFrame(p0)
p0_df = p0_df['p0_now'].map(abs).round(0)

df = pd.merge(s_nom, p0_df, how='inner', left_index=True, right_index=True)
df['loading_frac'] = (1 - (df['s_nom'] - df['p0_now']) / df['s_nom']).round(2)
df.sort_values(by='loading_frac', ascending=False).head()

,s_nom,p0_now,loading_frac
name,,,
L50,96.0,135.0,1.41
L51,96.0,135.0,1.41
L67,70.0,54.0,0.77
L9,76.0,53.0,0.70
L8,76.0,53.0,0.70
